# Code Generator

The requirement: use an Open Source model to generate high performance C++ code from Python code

To replicate this, you'll need to set up a HuggingFace endpoint as I do in the video. It's simple to do, and it's quite satisfying to see the results!

It's also an important part of your learning; this is the first example of deploying an open source model to be behind an API. We'll return to this in Week 8, but this should plant a seed in your mind for what's involved in moving open source models into production.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important - Pause Endpoints when not in use</h1>
            <span style="color:#900;">
            If you do decide to use HuggingFace endpoints for this project, you should stop or pause the endpoints when you are done to avoid accruing unnecessary running cost. The costs are very low as long as you only run the endpoint when you're using it. Navigate to the HuggingFace endpoint UI <a href="https://ui.endpoints.huggingface.co/">here,</a> open your endpoint, and click Pause to put it on pause so you no longer pay for it.  
Many thanks to student John L. for raising this.
<br/><br/>
In week 8 we will use Modal instead of HuggingFace endpoints; with Modal you only pay for the time that you use it and you should get free credits.
            </span>
        </td>
    </tr>
</table>

In [51]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
import platform

In [52]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [53]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [166]:
# Define the different actions available

prompt_options = ["Convert to C", "Add comments", "Write unit tests"]

system_prompts = {
    prompt_options[0]:  """
You are an assistant that reimplements Python code in high performance C++.
Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly.
""",
    
    prompt_options[1]:  """
You are an assistant that adds succinct comments and docstrings to Python code. Respond only with valid Python code.
""",
    
    prompt_options[2]:  """
You are an assistant that creates unit tests for Python code. Respond only with valid Python code.
"""
}

user_prompts = {
    prompt_options[0]: """
Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. 
Respond only with C++ code; do not explain your work other than a few comments.
Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n
""",
    
    prompt_options[1]:  """
Keep this Python code but insert appropriate comments and docstrings.
""",
    
    prompt_options[2]:  """
Create unit tests for this Python code.
"""
}

In [157]:
python_sample_options = ["Hello, World", "Calculate pi", "Kadane's Algorithm", "Sieve of Eratosthenes"]

python_code_samples = {
    python_sample_options[0]: """
import time

start_time = time.time()

print("Hello, world")

end_time = time.time()

print(f"Execution Time: {(end_time - start_time):.6f} seconds")
""",

    python_sample_options[1]:  """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
""",

    python_sample_options[2]:  """
# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
""",

    python_sample_options[3]:  """
import time
start_time = time.time()
stop_at=100_000_000
prime = [True] * (stop_at + 1)
p = 2

while p * p <= stop_at:
    # If prime[p] is True, then p is a prime
    if prime[p]:
        # Mark all multiples of p as non-prime
        for i in range(p * p, stop_at + 1, p):
            prime[i] = False
    p += 1

# Collect all prime numbers
primes = [p for p in range(2, stop_at + 1) if prime[p]]

end_time = time.time()
print("Maximum prime:, {:,}".format(primes[-1]))
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""
}

In [56]:
# Create a complete user prompt given descriptive text plus the python code to convert.
def create_user_prompt(user_prompt, python_code):
    return user_prompt + '\n' + python_code

# Create the list the GPT. Claude doesn't need this because it does not combine the system and user prompts.
def create_messages_for_gpt(system_prompt, user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [57]:
def write_cpp_file(filename_base, cpp_code):
    code_to_write = cpp.replace("```cpp","").replace("```","")
    with open(f"{filename_base}.cpp", "w") as f:
        f.write(code)

In [164]:
# This is where additional models can be easily added. 
# Just update the model_options list, add a streaming function, and update the call_llm function. 

model_options = ["GPT", "Claude"]
# model_options = ["GPT", "Claude", "CodeQwen"]
default_model = model_options[0]

def stream_gpt(system_prompt, user_prompt, python_code):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=create_messages_for_gpt(system_prompt, create_user_prompt(user_prompt, python_code)), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

def stream_claude(system_prompt, user_prompt, python_code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_prompt,
        messages=[{"role": "user", "content": create_user_prompt(user_prompt, python_code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

def call_llm(system_prompt, user_prompt, python_code, model):
    if model=="GPT":
        result = stream_gpt(system_prompt, user_prompt, python_code)
    elif model=="Claude":
        result = stream_claude(system_prompt, user_prompt, python_code)
    # elif model=="CodeQwen":
    #     result = stream_code_qwen(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far    

In [ ]:
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
code_gemma = "google/codegemma-7b-it"
CODE_QWEN_URL = "https://h1vdol7jxhje3mpn.us-east-1.aws.endpoints.huggingface.cloud"
CODE_GEMMA_URL = "https://c5hggiyqachmgnqg.us-east-1.aws.endpoints.huggingface.cloud"

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

def stream_code_qwen(python):
    tokenizer = AutoTokenizer.from_pretrained(code_qwen)
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(CODE_QWEN_URL, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)
    result = ""
    for r in stream:
        result += r.token.text
        yield result    

In [61]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

def execute_cpp(code):
    write_output(code)
    try:
        compile_result = subprocess.run(compiler_cmd[2], check=True, text=True, capture_output=True)
        run_cmd = ["./optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [62]:
def select_python_sample(python_sample):
    if python_sample in python_sample_options:
        return python_code_samples[python_sample]
    else:
        return next(iter(donedone.values()), "# Type in your Python program here")

In [63]:
import platform

VISUAL_STUDIO_2022_TOOLS = "C:\\Program Files\\Microsoft Visual Studio\\2022\\Community\\Common7\Tools\\VsDevCmd.bat"
VISUAL_STUDIO_2019_TOOLS = "C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\BuildTools\\Common7\\Tools\\VsDevCmd.bat"

simple_cpp = """
#include <iostream>

int main() {
    std::cout << "Hello";
    return 0;
}
"""

def run_cmd(command_to_run):
    try:
        run_result = subprocess.run(command_to_run, check=True, text=True, capture_output=True)
        return run_result.stdout if run_result.stdout else "SUCCESS"
    except:
        return ""

def c_compiler_cmd(filename_base):
    my_platform = platform.system()
    my_compiler = []

    try:
        with open("simple.cpp", "w") as f:
            f.write(simple_cpp)
            
        if my_platform == "Windows":
            if os.path.isfile(VISUAL_STUDIO_2022_TOOLS):
                if os.path.isfile("./simple.exe"):
                    os.remove("./simple.exe")
                compile_cmd = ["cmd", "/c", VISUAL_STUDIO_2022_TOOLS, "&", "cl", "simple.cpp"]
                if run_cmd(compile_cmd):
                    if run_cmd(["./simple.exe"]) == "Hello":
                        my_compiler = ["Windows", "Visual Studio 2022", ["cmd", "/c", VISUAL_STUDIO_2022_TOOLS, "&", "cl", f"{filename_base}.cpp"]]
        
            if not my_compiler:
                if os.path.isfile(VISUAL_STUDIO_2019_TOOLS):
                    if os.path.isfile("./simple.exe"):
                        os.remove("./simple.exe")
                    compile_cmd = ["cmd", "/c", VISUAL_STUDIO_2019_TOOLS, "&", "cl", "simple.cpp"]
                    if run_cmd(compile_cmd):
                        if run_cmd(["./simple.exe"]) == "Hello":
                            my_compiler = ["Windows", "Visual Studio 2019", ["cmd", "/c", VISUAL_STUDIO_2019_TOOLS, "&", "cl", f"{filename_base}.cpp"]]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
                
        elif my_platform == "Linux":
            if os.path.isfile("./simple"):
                os.remove("./simple")
            compile_cmd = ["g++", "simple.cpp", "-o", "simple"]
            if run_cmd(compile_cmd):
                if run_cmd(["./simple"]) == "Hello":
                    my_compiler = ["Linux", "GCC (g++)", ["g++", f"{filename_base}.cpp", "-o", f"{filename_base}" ]]
    
            if not my_compiler:
                if os.path.isfile("./simple"):
                    os.remove("./simple")
                compile_cmd = ["clang++", "simple.cpp", "-o", "simple"]
                if run_cmd(compile_cmd):
                    if run_cmd(["./simple"]) == "Hello":
                        my_compiler = ["Linux", "Clang++", ["clang++", f"{filename_base}.cpp", "-o", f"{filename_base}"]]
        
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    
        elif my_platform == "Darwin":
            if os.path.isfile("./simple"):
                os.remove("./simple")
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "simple", "simple.cpp"]
            if run_cmd(compile_cmd):
                if run_cmd(["./simple"]) == "Hello":
                    my_compiler = ["Macintosh", "Clang++", ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", f"{filename_base}", f"{filename_base}.cpp"]]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    except:
        my_compiler=[my_platform, "Unavailable", []]
        
    if my_compiler:
        return my_compiler
    else:
        return ["Unknown", "Unavailable", []]


In [167]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

model = default_model
selected_tab = prompt_options[0]

# Determine the C (C++, really) compiler to use based on the platform
compiler_cmd = c_compiler_cmd("optimized")

def any_tab_on_select(evt: gr.SelectData):
    global selected_tab
    selected_tab = evt.value

def reset_prompts():
    return system_prompts[selected_tab], user_prompts[selected_tab]

def change_python_sample(python_sample, python_code):
    if not python_sample == "Custom":
        if python_sample in python_sample_options:
            return python_code_samples[python_sample]
        else:
            return python_code
    else:
        return python_code

def change_python_sample_to_custom():
    return "Custom"

# Display the interface
with gr.Blocks(css=css) as ui:
    with gr.Tab(prompt_options[0]) as first_tab:
        gr.Markdown("# " + prompt_options[0])
        with gr.Group():
            with gr.Row():
                first_system_prompt_txt = gr.Textbox(label="System prompt", value=system_prompts[prompt_options[0]], lines=10, interactive=True )
                first_user_prompt_txt = gr.Textbox(label="User prompt", value=user_prompts[prompt_options[0]], lines=10, interactive=True )
            with gr.Row():
                first_reset_prompts_btn = gr.Button("Reset prompts")
        with gr.Row():
            with gr.Column():
                first_sample_program_rad = gr.Radio(python_sample_options + ["Custom"], label="Sample program", value=python_sample_options[0])
                first_python_code_txt = gr.Textbox(label="Python code:", value=python_code_samples[python_sample_options[0]], lines=10, interactive=True)
            with gr.Column():
                first_model_drp = gr.Dropdown(model_options, label="Select model", value=default_model, interactive=True)
                first_convert_btn = gr.Button("Convert code", interactive=True)
                first_cpp_txt = gr.Textbox(label="C++ code:", lines=10, interactive=True)
        with gr.Row():
            with gr.Column():
                with gr.Group():
                    first_python_run_btn = gr.Button("Run Python", interactive=True)
                    first_python_out_txt = gr.TextArea(label="Python result:", elem_classes=["python"])
            with gr.Column():
                with gr.Group():
                    if not compiler_cmd[1] == "Unavailable":
                        first_cpp_run_btn = gr.Button("Run C++")
                    else:
                        first_cpp_run_btn = gr.Button("No compiler to run C++", interactive=False)
                    first_cpp_out_txt = gr.TextArea(label="C++ result:", elem_classes=["cpp"])
                    first_architecture_rad = gr.Radio([compiler_cmd[0]], label="Architecture", interactive=False, value=compiler_cmd[0])
                    first_compiler_rad = gr.Radio([compiler_cmd[1]], label="Compiler", interactive=False, value=compiler_cmd[1])
            
    with gr.Tab(prompt_options[1]) as second_tab:
        gr.Markdown("# " + prompt_options[1])
        with gr.Group():
            with gr.Row():
                second_system_prompt_txt = gr.Textbox(label="System prompt", value=system_prompts[prompt_options[1]], lines=10, interactive=True )
                second_user_prompt_txt = gr.Textbox(label="User prompt", value=user_prompts[prompt_options[1]], lines=10, interactive=True )
            with gr.Row():
                second_reset_prompts_btn = gr.Button("Reset prompts")
        with gr.Row():
            with gr.Column():
                second_sample_program_rad = gr.Radio(python_sample_options + ["Custom"], label="Sample program", value=python_sample_options[1])
                second_python_code_txt = gr.Textbox(label="Python code:", value=python_code_samples[python_sample_options[1]], lines=10)
            with gr.Column():
                second_model_drp = gr.Dropdown(model_options, label="Select model", value=default_model)
                second_comment_btn = gr.Button("Comment code")
                second_python_code_comments_txt = gr.Textbox(label="Commented code", lines=20)

                    
    with gr.Tab(prompt_options[2]) as third_tab:
        gr.Markdown("# " + prompt_options[2])
        with gr.Group():
            with gr.Row():
                third_system_prompt_txt = gr.Textbox(label="System prompt", value=system_prompts[prompt_options[2]], lines=10, interactive=True )
                third_user_prompt_txt = gr.Textbox(label="User prompt", value=user_prompts[prompt_options[2]], lines=10, interactive=True )
            with gr.Row():
                third_reset_prompts_btn = gr.Button("Reset prompts")
        with gr.Row():
            with gr.Column():
                third_sample_program_rad = gr.Radio(python_sample_options + ["Custom"], label="Sample program", value=python_sample_options[1])
                third_python_code_txt = gr.Textbox(label="Python code:", value=python_code_samples[python_sample_options[1]], lines=10)
            with gr.Column():
                third_model_drp = gr.Dropdown(model_options, label="Select model", value=default_model)
                third_unit_test_btn = gr.Button("Create unit tests")
                third_python_unit_tests_txt = gr.Textbox(label="Unit tests", lines=20)

    first_tab.select(any_tab_on_select)
    second_tab.select(any_tab_on_select)
    third_tab.select(any_tab_on_select)
    
    first_reset_prompts_btn.click(reset_prompts, outputs=[first_system_prompt_txt, first_user_prompt_txt])
    second_reset_prompts_btn.click(reset_prompts, outputs=[second_system_prompt_txt, second_user_prompt_txt])
    third_reset_prompts_btn.click(reset_prompts, outputs=[third_system_prompt_txt, third_user_prompt_txt])

    first_sample_program_rad.input(change_python_sample, inputs=[first_sample_program_rad, first_python_code_txt], outputs=[first_python_code_txt])
    first_python_code_txt.input(change_python_sample_to_custom, inputs=[], outputs=[first_sample_program_rad])
    first_convert_btn.click(call_llm, inputs=[first_system_prompt_txt, first_user_prompt_txt, first_python_code_txt, first_model_drp], outputs=[first_cpp_txt])
    first_python_run_btn.click(execute_python, inputs=[first_python_code_txt], outputs=[first_python_out_txt])
    first_cpp_run_btn.click(execute_cpp, inputs=[first_cpp_txt], outputs=[first_cpp_out_txt])

    second_sample_program_rad.input(change_python_sample, inputs=[second_sample_program_rad, second_python_code_txt], outputs=[second_python_code_txt])
    second_python_code_txt.input(change_python_sample_to_custom, inputs=[], outputs=[second_sample_program_rad])
    second_comment_btn.click(call_llm, inputs=[second_system_prompt_txt, second_user_prompt_txt, second_python_code_txt, second_model_drp], outputs=[second_python_code_comments_txt])

    third_sample_program_rad.input(change_python_sample, inputs=[third_sample_program_rad, third_python_code_txt], outputs=[third_python_code_txt])
    third_python_code_txt.input(change_python_sample_to_custom, inputs=[], outputs=[second_sample_program_rad])
    third_unit_test_btn.click(call_llm, inputs=[third_system_prompt_txt, third_user_prompt_txt, third_python_code_txt, third_model_drp], outputs=[third_python_unit_tests_txt])
ui.launch(inbrowser=True)

In [152]:
import time

start_time = time.time()
    
print("Hello, world")
    
end_time = time.time()
    

print(f"Execution Time: {(end_time - start_time):.6f} seconds")